In [ ]:
import os
import json
import torch
import random
from google.colab import drive
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments, EarlyStoppingCallback
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim import AdamW

Montar disco

In [ ]:
drive.mount('/content/drive')
SPIDER_PATH = "/content/drive/My Drive/spider"

Mounted at /content/drive


Cargar entrenamiento

In [ ]:
def load_data(filepath):
    with open(filepath, "r") as f:
        return json.load(f)

train_data = load_data(f"{SPIDER_PATH}/train_spider.json")
val_data = load_data(f"{SPIDER_PATH}/dev.json")
test_data = load_data(f"{SPIDER_PATH}/test.json")

# Cargar metadata de esquemas
schema_metadata = load_data(f"{SPIDER_PATH}/tables.json")

# Función para obtener el esquema correcto
def get_schema_info(db_id, schema_metadata):
    for schema in schema_metadata:
        if schema["db_id"] == db_id:
            return json.dumps(schema)  # Convertir a string JSON para mejor manejo
    return ""

Cargar Metadata Esquemas

In [ ]:
def preprocess_spider(data, schema_metadata):
    inputs = []
    targets = []
    for item in data:
        question = item["question"]
        sql_query = item["query"]
        db_id = item["db_id"]
        schema_info = get_schema_info(db_id, schema_metadata)
        inputs.append(f"Schema: {schema_info} \n Translate to SQL: {question}")
        targets.append(sql_query)
    return inputs, targets

train_inputs, train_targets = preprocess_spider(train_data, schema_metadata)
val_inputs, val_targets = preprocess_spider(val_data, schema_metadata)
test_inputs, test_targets = preprocess_spider(test_data, schema_metadata)

Pre procesar datos

In [ ]:
def preprocess_spider(data, schema_metadata):
    inputs = []
    targets = []
    for item in data:
        question = item["question"]
        sql_query = item["query"]
        db_id = item["db_id"]
        schema_info = get_schema_info(db_id, schema_metadata)
        inputs.append(f"Schema: {schema_info} \n Translate to SQL: {question}")
        targets.append(sql_query)
    return inputs, targets

train_inputs, train_targets = preprocess_spider(train_data, schema_metadata)
val_inputs, val_targets = preprocess_spider(val_data, schema_metadata)

Tokenizar Datos

In [ ]:
MODEL_NAME = "SwastikM/bart-large-nl2sql"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_data(inputs, targets, tokenizer, max_input_len=512, max_target_len=128):
    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=max_input_len, return_tensors="pt")
    labels = tokenizer(targets, padding="max_length", truncation=True, max_length=max_target_len, return_tensors="pt").input_ids
    return model_inputs, labels

train_encodings, train_labels = tokenize_data(train_inputs, train_targets, tokenizer)
val_encodings, val_labels = tokenize_data(val_inputs, val_targets, tokenizer)
test_encodings, test_labels = tokenize_data(test_inputs, test_targets, tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

Dataset Personalizado

In [ ]:
class SpiderDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_dataset = SpiderDataset(train_encodings, train_labels)
val_dataset = SpiderDataset(val_encodings, val_labels)
test_dataset = SpiderDataset(test_encodings, test_labels)

Cargar Modelo

In [ ]:
model = BartForConditionalGeneration.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/1.70k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/406 [00:00<?, ?B/s]

Optimizador configuracion

In [ ]:
optimizer = AdamW(model.parameters(), lr=3e-5)

Scheduler con CosineAnnealingLR

In [ ]:
scheduler = CosineAnnealingLR(optimizer, T_max=10)

Parametrizacion del entrenamiento

In [ ]:
training_args = TrainingArguments(
    output_dir="/content/drive/My Drive/bart_nl2sql_finetuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=8,
    weight_decay=0.05,
    warmup_steps=500,
    logging_dir="/content/drive/My Drive/logs",
    logging_steps=50,
    save_total_limit=2,
    num_train_epochs=20,  # Aumentado a 20 para mejor convergencia
    fp16=True,  # Activar entrenamiento en precisión mixta
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_on_each_node=True,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Scheduler para reducir el learning rate

Early Stop

In [ ]:
# Entrenador con Early Stopping
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    optimizers=(optimizer, scheduler),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

Entrenador

Aplicar Entrenamiento

In [ ]:
trainer.train()

<ipython-input-7-6876e9938160>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-7-6876e9938160>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item["labels"] = torch.tensor(self.labels[idx])


Epoch,Training Loss,Validation Loss
1,0.424400,0.290676
2,0.150000,0.260993
3,0.104800,0.277349
4,0.084900,0.274211
5,0.070600,0.292213


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
<ipython-input-7-6876e9938160>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-7-6876e9938160>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTenso

TrainOutput(global_step=550, training_loss=0.4420294713973999, metrics={'train_runtime': 602.5899, 'train_samples_per_second': 232.33, 'train_steps_per_second': 3.618, 'total_flos': 3.792433053696e+16, 'train_loss': 0.4420294713973999, 'epoch': 5.0})

Guardar Modelo

In [ ]:
model.save_pretrained("/content/drive/My Drive/bart_nl2sql_finetuned/final_model3")
tokenizer.save_pretrained("/content/drive/My Drive/bart_nl2sql_finetuned/final_model3")




('/content/drive/My Drive/bart_nl2sql_finetuned/final_model3/tokenizer_config.json',
 '/content/drive/My Drive/bart_nl2sql_finetuned/final_model3/special_tokens_map.json',
 '/content/drive/My Drive/bart_nl2sql_finetuned/final_model3/vocab.json',
 '/content/drive/My Drive/bart_nl2sql_finetuned/final_model3/merges.txt',
 '/content/drive/My Drive/bart_nl2sql_finetuned/final_model3/added_tokens.json',
 '/content/drive/My Drive/bart_nl2sql_finetuned/final_model3/tokenizer.json')

Metricas de Evaluacion

In [ ]:
# Evaluar modelo y mostrar métricas
eval_results = trainer.evaluate()
print("Resultados de Evaluación:")
for key, value in eval_results.items():
    print(f"{key}: {value}")

<ipython-input-7-6876e9938160>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-7-6876e9938160>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item["labels"] = torch.tensor(self.labels[idx])


Resultados de Evaluación:
eval_loss: 0.2609925866127014
eval_runtime: 5.0071
eval_samples_per_second: 206.508
eval_steps_per_second: 25.963
epoch: 5.0


In [ ]:
test_results = trainer.evaluate(test_dataset)
print("\nResultados de Evaluación en Test:")
for key, value in test_results.items():
    print(f"{key}: {value}")

<ipython-input-7-6876e9938160>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-7-6876e9938160>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item["labels"] = torch.tensor(self.labels[idx])



Resultados de Evaluación en Test:
eval_loss: 0.3242042660713196
eval_runtime: 10.5851
eval_samples_per_second: 202.833
eval_steps_per_second: 25.413
epoch: 5.0


Prueba de preguntas aleatorias